<a href="https://colab.research.google.com/github/dawidplotka-analyst/A-B-Testing-/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import google.colab.auth
from google.colab import drive

#dostęp do mojego drive
drive.mount('/content/drive')

# Authenticate user to Google Cloud
google.colab.auth.authenticate_user()

#importuje baze danych z BigQuery:
baza = bigquery.Client(project="data-analytics-mate")

#za pomocą kodu SQL poniżej wypisanego, przygotowuję sobie gotową tabele, na której będę pracował:

sql = """
with
session_info AS (
  select
  s.date,
  s.ga_session_id,
  sp.country,
  sp.device,
  sp.continent,
  sp.channel,
  ab.test,
  ab.test_group
  from `DA.ab_test` AS ab
  JOIN `DA.session` AS s
USING(ga_session_id)
JOIN `DA.session_params` AS sp
USING(ga_session_id)
),
session_with_order AS (
 select
  session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(distinct o.ga_session_id) AS session_with_orders
FROM `DA.order` AS o
JOIN session_info
USING(ga_session_id)
GROUP BY
 session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
),
events AS (
SELECT
session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name,
  COUNT(distinct ep.ga_session_id) AS event_cnt
  FROM `DA.event_params` AS ep
  JOIN session_info
  USING(ga_session_id)
  GROUP BY
  session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name
),
sessions AS (


SELECT
  session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(distinct session_info.ga_session_id) AS session_cnt
  FROM session_info
  GROUP BY
    session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
),
account AS(
SELECT
  session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(distinct acs.ga_session_id) AS new_account_cnt
  FROM `DA.account_session` AS acs
  JOIN session_info
  USING(ga_session_id)
  GROUP BY
    session_info.date,
  session_info.ga_session_id,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
)

SELECT
  session_with_order.date,
  session_with_order.ga_session_id,
  session_with_order.country,
  session_with_order.device,
  session_with_order.continent,
  session_with_order.channel,
  session_with_order.test,
  session_with_order.test_group,
  'session_with_orders' AS event_name,
  session_with_order.session_with_orders AS value
FROM session_with_order


UNION ALL
SELECT
  events.date,
  events.ga_session_id,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  events.event_cnt AS value
  FROM events


  UNION ALL
SELECT
  sessions.date,
  sessions.ga_session_id,
  sessions.country,
  sessions.device,
  sessions.continent,
  sessions.channel,
  sessions.test,
  sessions.test_group,
  'session' AS event_name,
  session_cnt AS value
FROM sessions


 UNION ALL
SELECT
  account.date,
  account.ga_session_id,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_account_cnt AS value
  FROM account


"""

tabela_bazowa_ABtest = baza.query(sql).to_dataframe()
print(tabela_bazowa_ABtest.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         date  ga_session_id               country   device continent  \
0  2020-11-01     2146075826              Pakistan   mobile      Asia   
1  2020-11-02     5068368283  United Arab Emirates   mobile      Asia   
2  2020-11-04     1685197615                Greece  desktop    Europe   
3  2020-11-05     9410409718           El Salvador  desktop  Americas   
4  2020-11-05     7781023244            Costa Rica  desktop  Americas   

          channel  test  test_group           event_name  value  
0  Organic Search     2           2  session_with_orders      1  
1     Paid Search     2           2  session_with_orders      1  
2     Paid Search     2           2  session_with_orders      1  
3   Social Search     2           1  session_with_orders      1  
4          Direct     2           1  session_with_orders      1  


# Analiza Statystyczna Testów A/B (Python)
Cel: Automatyzacja obliczeń istotności statystycznej dla kluczowych metryk konwersji przy użyciu testu $Z$ dla proporcji.

Logika kodu:
- Agregacja danych: Dane zostały pogrupowane według grup (Control vs Test) oraz wybranych wymiarów.Pętla po metrykach: Zamiast liczyć każdą metrykę osobno, stworzyłem pętlę, która przechodzi przez listę zdefiniowanych wskaźników 'add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account'. Dzięki temu skrypt jest skalowalny – dodanie nowej metryki wymaga tylko dopisania jej do listy.
- Obliczenia statystyczne: Wykorzystałem bibliotekę statsmodels, aby obliczyć z-score i p-value.Przyjąłem poziom istotności $\alpha = 0,05$. Jeśli $p$-value jest mniejsze niż $0,05$, uznajemy zmianę za statystycznie istotną (Significant = True).
- Wynik: Skrypt generuje tabelę wynikową z konwersją dla obu grup oraz werdyktem statystycznym.

In [ ]:
#tutaj stworzę pętlę dla Z-test
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# 1. Definicja metryk do analizy
metrics_list = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']

results = []

# 2. Pętla po unikalnych testach
for t in tabela_bazowa_ABtest['test'].unique():
    test_data = tabela_bazowa_ABtest[tabela_bazowa_ABtest['test'] == t]

    # Mianownik (łączna liczba unikalnych sesji dla grup Control i Test)
    n_control = test_data[(test_data['test_group'] == 1) & (test_data['event_name'] == 'session')]['ga_session_id'].nunique()
    n_test = test_data[(test_data['test_group'] == 2) & (test_data['event_name'] == 'session')]['ga_session_id'].nunique()

    for m in metrics_list:
        # Licznik (liczba sesji, w których wystąpił dany event)
        count_control = test_data[(test_data['test_group'] == 1) & (test_data['event_name'] == m)]['ga_session_id'].nunique()
        count_test = test_data[(test_data['test_group'] == 2) & (test_data['event_name'] == m)]['ga_session_id'].nunique()

        # Statystyka Z-test
        z_stat, p_val = proportions_ztest([count_test, count_control], [n_test, n_control])

        # CR i Uplift
        cr_control = count_control / n_control
        if n_control > 0 else 0
        cr_test = count_test / n_test
        if n_test > 0 else 0

        results.append({
            'test_number': t,
            'metric': m,
            'numerator_control': count_control,
            'denominator_control': n_control,
            'numerator_test': count_test,
            'denominator_test': n_test,
            'conversion_rate_control': cr_control,
            'conversion_rate_test': cr_test,
            'z_stat': z_stat,
            'p_value': p_val,
            'significant': p_val < 0.05
        })

# 3. Generowanie tabeli końcowej
final_table = pd.DataFrame(results)

# Export do pliku pod Tableau
final_table.to_csv('wyniki_ab_test.csv', index=False)

# Podgląd tabeli
final_table

,test_number,metric,numerator_control,denominator_control,numerator_test,denominator_test,conversion_rate_control,conversion_rate_test,z_stat,p_value,significant
0,2,add_payment_info,1096,50637,1125,50244,0.021644,0.022391,0.807895,0.419151,False
1,2,add_shipping_info,2194,50637,2136,50244,0.043328,0.042513,-0.638943,0.522860,False
2,2,begin_checkout,2195,50637,2137,50244,0.043348,0.042532,-0.638682,0.523030,False
3,2,new account,4165,50637,4184,50244,0.082252,0.083274,0.588793,0.556000,False
4,1,add_payment_info,964,45362,983,45193,0.021251,0.021751,0.518551,0.604074,False
5,1,add_shipping_info,1860,45362,1992,45193,0.041003,0.044078,2.291930,0.021910,True
6,1,begin_checkout,1862,45362,1992,45193,0.041048,0.044078,2.258498,0.023915,True
7,1,new account,3823,45362,3681,45193,0.084278,0.081451,-1.542883,0.122859,False
8,4,add_payment_info,1850,105079,1841,105141,0.017606,0.017510,-0.167535,0.866949,False
9,4,add_shipping_info,2661,105079,2667,105141,0.025324,0.025366,0.061455,0.950996,False


# 📊 Wizualizacja i Analiza Wyników (Tableau) Cel wizualizacji:
Stworzenie interaktywnego narzędzia do monitorowania wyników testów A/B, które pozwala na błyskawiczne odróżnienie szumu statystycznego od realnych sukcesów biznesowych.

### Co zostało zrobione:
Integracja danych: Połączono dane surowe z wynikami obliczeń statystycznych wykonanych w Pythonie.Interaktywny Layout: Zastosowano system kontenerów z blokadą wysokości (Fixed Height), co zapewnia czytelność dashboardu niezależnie od liczby wybranych filtrów.

### Wizualizacja Istotności:
Wykorzystano kodowanie kolorystyczne, aby od razu wskazać metryki, w których zmiana jest istotna statystycznie ($p$-value < 0,05).

🔗 [Kliknij tutaj, aby otworzyć interaktywny Dashboard w Tableau](https://public.tableau.com/views/ConversionRateComparison/ConversionRateComparison?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

# 💡Kluczowe Wnioski:
### Sukces Testu nr 1:
Odnotowano statystycznie istotny wzrost konwersji w krokach Shipping Info oraz Checkout. Zmiany w UX na tym etapie skutecznie popchnęły użytkowników dalej w dół lejka sprzedażowego.

### Brak wpływu na rejestracje:
Żaden z testów nie wpłynął na wskaźnik zakładania nowych kont (New Accounts). Oznacza to, że optymalizacja checkoutu nie rozwiązuje problemu konwersji na etapie logowania/rejestracji.

### Testy 2, 3 i 4:
Mimo drobnych różnic w procentach, analiza wykazała brak istotności statystycznej. Zmiany w tych testach nie dają gwarancji powtarzalnych wyników w przyszłości.

# 🎯Rekomendacje Biznesowe:
**Wdrożenie (Test 1):** Rekomenduję pełną implementację zmian z Testu nr 1, ponieważ realnie zwiększają one efektywność procesu płatności.

**Zaniechanie wdrożeń (Testy 2-4):** Odradzam wdrażanie zmian z pozostałych testów, aby uniknąć zbędnych kosztów technicznych przy braku zysków biznesowych.

**🚀Nowy Kierunek:** Proponuję kolejny test A/B skoncentrowany wyłącznie na procesie New Accounts, aby sprawdzić, co blokuje użytkowników przed finalną rejestracją.